In [1]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_wine
import pandas as pd

In [2]:
model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

In [3]:
df_alert_date = pd.read_csv("./訓練資料集_first/train_x_alert_date.csv")
df_custinfo = pd.read_csv("./訓練資料集_first/public_train_x_custinfo_full_hashed.csv")
df_dp = pd.read_csv("./訓練資料集_first/public_train_x_dp_full_hashed.csv")
df_answer = pd.read_csv("./訓練資料集_first/train_y_answer.csv")
data = df_alert_date.merge(df_custinfo, how= 'left', on = 'alert_key')
data = data.merge(df_dp['cust_id'].value_counts(), how = 'left', left_on='cust_id', right_index=True)
data = data.merge(df_answer, how= 'left', on = 'alert_key')
data

,alert_key,date,cust_id_x,risk_rank,occupation_code,total_asset,AGE,cust_id_y,sar_flag
0,171189,0,972ee157e63316e8a50dd489bc93730a3ee8a8959d5c6b...,1,17.0,375576.0,4,284.0,0
1,171202,0,a10ab33f90926fb18d7bb5e78034d7f04a1fbed95b7951...,3,12.0,2717416.0,2,217.0,0
2,171599,0,3433ecc068ed1c9e2f5037cab5d42d7b901e9bd624c1fb...,1,12.0,326517.0,4,205.0,0
3,171737,0,a0861608615a9365d90f4ba0a813c0ea0471987f925c8b...,3,14.0,1014759.0,4,504.0,0
4,171142,0,a39fea9aec90969fe66a2b2b4d1b86368a2d38e8b8d4bf...,3,12.0,241719.0,3,25183.0,0
...,...,...,...,...,...,...,...,...,...
23901,352132,364,c49b33d1fde790ec03584672903f296e486615adfdc989...,1,19.0,3218731.0,3,157.0,0
23902,352125,364,0c2dc5fedc3689abf5ff4be14fe8fea8d23d22068297c2...,3,19.0,928963.0,3,20.0,0
23903,352080,364,1a93961c8fa830a1f32f5316b44f4964d65397f0311b11...,2,9.0,69080.0,6,71.0,0
23904,352075,364,9586b80d3ad1d74c0a1efe792cae2ecd073243fb01ed33...,3,3.0,262604.0,6,474.0,0


In [4]:
X_train, X_test, y_train, y_test = train_test_split(data[['risk_rank', 'occupation_code', 'total_asset', 'AGE', 'cust_id_y']], data['sar_flag'], test_size=0.3, random_state=1)

In [5]:
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='mlogloss', gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_to_onehot=4, max_delta_step=0, max_depth=6,
              max_leaves=0, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [10]:
from sklearn.metrics import classification_report
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=['Normal','SAR']))

              precision    recall  f1-score   support

      Normal       0.99      1.00      0.99      7097
         SAR       0.11      0.01      0.02        75

    accuracy                           0.99      7172
   macro avg       0.55      0.51      0.51      7172
weighted avg       0.98      0.99      0.98      7172



In [8]:
df_public_alert_date = pd.read_csv("./訓練資料集_first/public_x_alert_date.csv")
public_data = df_public_alert_date.merge(df_custinfo, how= 'left', on = 'alert_key')
public_data = public_data.merge(df_dp['cust_id'].value_counts(), how = 'left', left_on='cust_id', right_index=True)
public_data = public_data.merge(df_answer, how= 'left', on = 'alert_key')
public_data

,alert_key,date,cust_id_x,risk_rank,occupation_code,total_asset,AGE,cust_id_y,sar_flag
0,352342,365,7629f010080267f680a4ca9c6cac71c4d4d86933dcc7ed...,1,17.0,506366.0,3,864.0,NaN
1,352866,365,e50deb3f2f87bf45e04940303f80c96929c836b0b03f54...,1,12.0,309830.0,4,55.0,NaN
2,352696,365,c6def618ad861703c025be4f41bdf7569310228ae93aef...,1,2.0,444392.0,3,339.0,NaN
3,352330,365,96d70b798f211393cfbcb83a8acd19f3e8c9c5c205960a...,1,19.0,143930.0,4,131.0,NaN
4,352683,365,40fa92acb89d1ebb319f0f200d31a5fcbdfcbd8ad432d7...,3,17.0,2112.0,2,333.0,NaN
...,...,...,...,...,...,...,...,...,...
1840,364472,393,bdda3ef79c5f052fd281c9f28f50a25977444eceaf9751...,3,19.0,2390889.0,4,760.0,NaN
1841,364788,393,0ef49d08b4af68e9cb2b2aa63198699ee7dc7b5e1c4550...,2,19.0,28681.0,2,155.0,NaN
1842,364673,393,f8ce663f765e2459578d285a032a16046fba0b2bba8e12...,1,19.0,213486.0,6,339.0,NaN
1843,364626,393,1729de666fbfd3bd42133e8226f5ade5709ef141d8e0b7...,1,19.0,2913.0,2,182.0,NaN


In [14]:
y_pred_public = model.predict(public_data[['risk_rank', 'occupation_code', 'total_asset', 'AGE', 'cust_id_y']])
pd.DataFrame({"alert_key" : public_data["alert_key"]})

,alert_key
0,352342
1,352866
2,352696
3,352330
4,352683
...,...
1840,364472
1841,364788
1842,364673
1843,364626
